Start: required imports

In [ ]:
import numpy as np
import pickle
import pandas as pd
from functools import partial
import glob
import seaborn as sbs
import matplotlib.pyplot as plt
import ioh

from scipy.stats import kendalltau, rankdata

font = {'size'   : 24}

plt.rc('font', **font)
from mpl_toolkits.axes_grid1 import make_axes_locatable
#Font-requirement for GECCO
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
from umap import UMAP

In [ ]:
from copy import copy

### Calculate scale factors

Perform random sampling on all functions to determine possible scale-factors

In [ ]:
np.random.seed(42)
max_vals = np.zeros((24,39))
mean_vals = np.zeros((24,39))
min_vals = np.zeros((24,39))
for dim in range(2,41):
    x_temp = np.random.uniform(low=-5,high=5, size=(50000,dim))
    for fid in range(1,25):
        f = ioh.get_problem(fid, instance=1, dimension=dim)
        vals = np.array([f(x) for x in x_temp])
        max_vals[fid-1, dim-2] = np.max(vals) - f.optimum.y
        mean_vals[fid-1, dim-2] = 10**np.mean(np.log10(np.clip(vals - f.optimum.y, 1e-10,1e20)))
        min_vals[fid-1, dim-2] = np.min(vals) - f.optimum.y
 

Make sure taking log doesn't result in NA

In [ ]:
min_vals = np.clip(min_vals, 1e-12, 1e20)

Plot the scale factors

In [ ]:
sf_temp = (np.log10(min_vals)+np.log10(mean_vals) / 2)+8

In [ ]:
dt_sf = pd.DataFrame(sf_temp)

In [ ]:
dt_sf['fid'] = range(1,25)

In [ ]:
dt_plot = dt_sf.melt(id_vars=['fid'])

In [ ]:
plt.figure(figsize=(16,7))
sbs.lineplot(dt_plot, x='variable', y='value', hue='fid', palette='viridis', legend=False)
plt.xlabel("Dimension")
plt.xticks([0,3,8,13,18,23,28,33,38], np.array([0,3,8,13,18,23,28,33,38])+2)
plt.ylabel("Log f(x)")
plt.tight_layout()
plt.savefig("Figures/Scale_factors.pdf")

In [ ]:
scale_factors = (np.log10(min_vals)+np.log10(mean_vals) / 2)+8

In [ ]:
scale_factors_rounded = np.round(np.median(scale_factors,axis=1), 1)

## Main function class

Make use of the scale-factors above and implement the function generator

In [ ]:
scale_factors = [11. , 17.5, 12.3, 12.6, 11.5, 15.3, 12.1, 15.3, 15.2, 17.4, 13.4,
       20.4, 12.9, 10.4, 12.3, 10.3,  9.8, 10.6, 10. , 14.7, 10.7, 10.8,
        9. , 12.1]

In [ ]:
class ManyAffine():
    def __init__(self, weights, instances, opt_loc=1, dim = 5, sf_type = 'hc'):
        self.weights = weights / np.sum(weights)
        self.fcts = [ioh.get_problem(fid, int(iid), dim) for fid, iid in zip(range(1,25), instances)]
        self.opts = [f.optimum.y for f in self.fcts]
        if sf_type == 'hc':
            self.scale_factors = scale_factors
        elif sf_type == 'mean':
            self.scale_factors = np.log10(mean_vals[:,dim-2])+8
        elif sf_type == 'max':
            self.scale_factors = np.log10(max_vals[:,dim-2])+8
        elif sf_type == 'min':
            self.scale_factors = np.log10(min_vals[:,dim-2])+8
        elif sf_type == 'min_max':
            self.scale_factors = (np.log10(min_vals[:,dim-2])+np.log10(mean_vals[:,dim-1]) / 2)+8
        else:
            self.scale_factors = np.ones(24)*10
        if type(opt_loc) == int:
            self.opt_x = self.fcts[opt_loc].optimum.x
        else:
            self.opt_x = opt_loc

    def __call__(self, x):
        raw_vals = np.array([ np.clip(f(x+f.optimum.x - self.opt_x)-o,1e-12,1e20) for f, o in zip(self.fcts, self.opts)])
        weighted = (np.log10(raw_vals)+8)/self.scale_factors * self.weights
        return 10**(10*np.sum(weighted)-8)

## Visualizations of 2D landscapes

In [ ]:
def create_vary_normalization(seed = None):
    dim = 2
    X, Y = np.meshgrid(np.arange(-5,5,0.1), np.arange(-5,5,0.1))
    Z = np.zeros(X.shape)
    
    fig, axs = plt.subplots(1,5, sharey=True, sharex=True, figsize=(32,9),
                        gridspec_kw={'hspace': 0, 'wspace': 0})
    if seed is not None:
        np.random.seed(seed)
    weights = np.clip(np.random.uniform(size=24, low=-1),0,1)
    iids = np.random.randint(1,100, size=24)
    opt_loc = np.random.uniform(size=(5,2), low=-5, high=5)
    sf_methods = ['mean', 'max', 'minmax', 'min', 'equal']
    for idx, ax in enumerate(axs):
        f_new = ManyAffine(weights, iids, opt_loc[1,:], dim=2, sf_type=sf_methods[idx])
        
        for idx1 in range(100):
            for idx2 in range(100):
                Z[idx1, idx2] = np.log10(f_new([X[idx1, idx2], Y[idx1, idx2]]))
        im = ax.contourf(X, Y, Z, 40, vmin = -4, vmax = 4);
        
        ax.scatter(f_new.opt_x[0], f_new.opt_x[1], c='r', s=25, linewidths=25, alpha=0.5)
        ax.set_xlabel(sf_methods[idx])


        if idx == 4:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')

    plt.tight_layout()
    plt.savefig(f"Figures/Example_scalefactors_S{seed}.pdf")
    plt.show()

In [ ]:
def create_vary_instance(seed = None):
    dim = 2
    X, Y = np.meshgrid(np.arange(-5,5,0.1), np.arange(-5,5,0.1))
    Z = np.zeros(X.shape)
    
    fig, axs = plt.subplots(1,5, sharey=True, sharex=True, figsize=(32,9),
                        gridspec_kw={'hspace': 0, 'wspace': 0})
    if seed is not None:
        np.random.seed(seed)
    weights = np.clip(np.random.uniform(size=24, low=-1),0,1)
    iids = np.random.randint(1,100, size=24)
    opt_loc = np.random.uniform(size=(5,2), low=-5, high=5)
    for idx, ax in enumerate(axs):
        f_new = ManyAffine(weights, iids, opt_loc[idx,:], dim=2)
        
        for idx1 in range(100):
            for idx2 in range(100):
                Z[idx1, idx2] = np.log10(f_new([X[idx1, idx2], Y[idx1, idx2]]))
        im = ax.contourf(X, Y, Z, 40, vmin = -4, vmax = 4);
        
        ax.scatter(f_new.opt_x[0], f_new.opt_x[1], c='r', s=25, linewidths=25, alpha=0.5)

        if idx == 4:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')

    plt.tight_layout()
    plt.savefig(f"Figures/Example_opt_loc_S{seed}.pdf")
    plt.show()

In [ ]:
for s in range(10):
    create_vary_instance(s)

In [ ]:
for s in range(10):
    create_vary_normalization(s)

In [ ]:
def create_vary_sampling(seed = None):
    dim = 2
    X, Y = np.meshgrid(np.arange(-5,5,0.1), np.arange(-5,5,0.1))
    Z = np.zeros(X.shape)
    
    fig, axs = plt.subplots(1,5, sharey=True, sharex=True, figsize=(32,9),
                        gridspec_kw={'hspace': 0, 'wspace': 0})
    if seed is not None:
        np.random.seed(seed)
    weights = np.clip(np.random.uniform(size=24, low=0),0,1)
    iids = np.random.randint(1,100, size=24)
    opt_loc = np.random.uniform(size=(5,2), low=-5, high=5)
    sf_methods = ['mean', 'min', 'max', 'minmax', 'equal']
    thresholds = [0,0.4,0.55,0.7,0.85]
    for idx, ax in enumerate(axs):
        weights_new = np.zeros(24)
        weights_new[weights>=thresholds[idx]] = weights[weights>=thresholds[idx]]-thresholds[idx]
        
        f_new = ManyAffine(weights_new, iids, opt_loc[1,:], dim=2)#, sf_type=sf_methods[idx])
        
        for idx1 in range(100):
            for idx2 in range(100):
                Z[idx1, idx2] = np.log10(f_new([X[idx1, idx2], Y[idx1, idx2]]))
        im = ax.contourf(X, Y, Z, 40, vmin = -4, vmax = 4);
        
        ax.scatter(f_new.opt_x[0], f_new.opt_x[1], c='r', s=25, linewidths=25, alpha=0.5)
        ax.set_xlabel(f"T={thresholds[idx]}")


        if idx == 4:
            divider = make_axes_locatable(ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            fig.colorbar(im, cax=cax, orientation='vertical')

    plt.tight_layout()
    plt.savefig(f"Figures/Example_weightsampling_S{seed}.pdf")
    plt.show()

In [ ]:
for s in range(10):
    create_vary_sampling(s)

In [ ]:
def create_opt_loc_plot(seed=None):
    opts = np.zeros((1000,2))
    if seed is not None:
        np.random.seed(seed)
    for i in range(1000):
        weights = np.clip(np.random.uniform(size=24, low=0),0,1)
        iids = np.random.randint(1,100, size=24)
        opt_loc = np.random.randint(24)
        f_new = ManyAffine(weights, iids, opt_loc, dim=2)
        opts[i] = f_new.opt_x

    plt.figure(figsize=(12,12))
    plt.scatter(opts[:,0],opts[:,1], s=100)
    plt.axhline(-4)
    plt.axhline(4)
    plt.axvline(4)
    plt.axvline(-4)
    plt.axhline(-5, c='r')
    plt.axhline(5, c='r')
    plt.axvline(5, c='r')
    plt.axvline(-5, c='r')
    plt.xticks([-5,-4,-2,0,2,4,5])
    plt.yticks([-5,-4,-2,0,2,4,5])
    plt.grid()
    plt.tight_layout()
    plt.savefig(f"Figures/opt_loc_2D.pdf")
    

In [ ]:
create_opt_loc_plot()

## ELA calculation

In [ ]:
dt_ela = pd.DataFrame()
for f in glob.glob("/mnt/d/Many_Affine/ELA2/*"):
    dt_temp = pd.read_csv(f, index_col=0)
    dt_temp = pd.DataFrame(dt_temp.mean()).transpose()
    basename = f.split('/')[-1][:-4]
    if basename.startswith('F'):
        dt_temp['kind'] = 'BBOB'
    else:
        dt_temp['kind'] = 'Affine'
    dt_temp['dim'] = int(basename.split('_')[-1][:-1])
    dt_temp['fid'] = f.split('/')[-1][:-4]
    dt_ela = dt_ela.append(dt_temp)

In [ ]:
dt_ela.to_csv("full_ela_5ids.csv")

## Setup performance data collection

This part determines the used settings for the tested functions

In [ ]:
np.random.seed(42)

In [ ]:
weights = np.random.uniform(size=(1000, 24))

In [ ]:
for idx, w in enumerate(weights):
    threshold = np.min([0.85, np.sort(w)[-3]])
    w[w<threshold] = 0
    w[w>=threshold] = w[w>=threshold]-threshold
    weights[idx] = w/np.sum(w)

In [ ]:
pd.DataFrame(weights).to_csv("weights.csv")

In [ ]:
iids = np.random.randint(1,100, size=(1000,24))

In [ ]:
pd.DataFrame(iids).to_csv("iids.csv")

In [ ]:
opt_locs = np.random.uniform(size=(1000, 5), low=-5, high=5)

In [ ]:
np.array(pd.DataFrame(opt_locs).iloc[0])

In [ ]:
pd.DataFrame(opt_locs).to_csv("opt_locs.csv")

## Analysis of performance

In [ ]:
dt_perf = pd.DataFrame()
for f in glob.glob("/mnt/d/Many_Affine/csv/auc/*"):
    dt_temp = pd.read_csv(f, index_col=0)
    basename = f.split('/')[-1][:-4]
    if basename.startswith('F'):
        dt_temp['kind'] = 'BBOB'
        dt_temp['fid'] = int(basename.split('_')[0][1:])
        dt_temp['iid'] = int(basename.split('_')[1][1:])
    else:
        dt_temp['kind'] = 'Affine'
        dt_temp['fid'] = int(basename.split('_')[0])
        dt_temp['iid'] = 0
    dt_temp['dim'] = int(basename.split('_')[-1][:-1])
#     dt_temp['fid'] = f.split('/')[-1][:-4]
    dt_perf = dt_perf.append(dt_temp.tail(1))

In [ ]:
dt_perf.to_csv("auc_performance.csv")

In [ ]:
dt_perf2d = dt_perf[(dt_perf['dim'] == 5)]

In [ ]:
dt_ranks = pd.DataFrame()
for r in range(1,6):
    dt_best = dt_perf2d.sort_values('auc', ascending=False).groupby(['kind', 'fid', 'iid']).apply(pd.DataFrame.head, n=r).reset_index(drop=True)
    dt_second = dt_best.groupby(['kind', 'fid', 'iid']).apply(pd.DataFrame.tail, n=1).reset_index(drop=True)
    dt_second['rank'] = r
    dt_ranks = dt_ranks.append(dt_second)

In [ ]:
for kind in ['BBOB', 'Affine']:
    plt.figure(figsize=(16,9))
    ax = sbs.histplot(data=dt_ranks[dt_ranks['kind'] == kind].sort_values('ID'), x="rank", hue="ID", multiple="stack", discrete=True, stat='proportion')
    sbs.move_legend(ax, 'upper center', ncol=3, bbox_to_anchor=(0.5,1.3), title='Algorithm Name')
    plt.yticks(np.arange(0,0.21,0.02), [f"{x:.1f}" for x in np.arange(0,0.21,0.02)*5])
    plt.ylim(0,0.2)
    plt.xlabel("AUC-based rank")
    plt.tight_layout()
    plt.savefig(f"Figures/Alg_rank_auc_{kind}_5D.pdf")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16,9), sharey=True)
kinds = ['BBOB', 'Affine']
for idx, ax in enumerate(axs.flatten()):
    sbs.histplot(ax=ax, data=dt_ranks[dt_ranks['kind'] == kinds[idx]].sort_values('ID'), x="rank", hue="ID", multiple="stack", discrete=True, stat='proportion', legend=(idx == 1))
    # if idx == 1:
        # sbs.move_legend(ax, 'lower center', ncol=3, bbox_to_anchor=(0,1), title='Algorithm Name')
    ax.set_yticks(np.arange(0,0.21,0.02), [f"{x:.1f}" for x in np.arange(0,0.21,0.02)*5])
    ax.set_xticks(range(1,6), range(1,6), fontsize=34)
    ax.set_ylim(0,0.2)
    ax.set_xlim(0.5,5.5)

    ax.set_xlabel(f"Rank ({kinds[idx]})", fontsize=40)
    ax.set_ylabel("Proportion", fontsize=40)
plt.tight_layout()
plt.savefig(f"Figures/Alg_rank_auc_both_5D.pdf")

# Merging data

### Merging

Full dataframe containing:
- function identifier
- ELA features
- weights used to create the function
- location of optima used
- instance numbers used
- auc performance after max budget (for each of 5 algorithms)

In [ ]:
weights = pd.read_csv("weights.csv", index_col=0)

In [ ]:
opt_locs = pd.read_csv("opt_locs.csv", index_col=0)

In [ ]:
iids = pd.read_csv("iids.csv", index_col=0)

In [ ]:
dt_ela = pd.read_csv("full_ela_5ids.csv", index_col=0)

In [ ]:
dt_perf = pd.read_csv("auc_performance.csv", index_col=0)

In [ ]:
def get_all_info_affine(fid, dim):
    ela_dict = dict(dt_ela[dt_ela['fid'] == f"{fid}_{dim}D"][relevant_columns].iloc[0])
    weight_dict = {f'W{idx}' : x for idx, x in enumerate(weights.iloc[fid])}
    iid_values = {f'I{idx}' : x for idx, x in enumerate(iids.iloc[fid])}
    opt_loc = {f'x{idx}' : x for idx, x in enumerate(opt_locs.iloc[fid][:dim])}
    dt_perf_sub = dt_perf[(dt_perf['kind'] == 'Affine') & (dt_perf['fid'] == fid) & (dt_perf['dim'] == dim)]
    perf_dict = {x:y for x,y in zip(dt_perf_sub['ID'], dt_perf_sub['auc'])}

    dict_all = {}
    dict_all.update(ela_dict)
    dict_all.update(weight_dict)
    dict_all.update(iid_values)
    dict_all.update(opt_loc)
    dict_all.update(perf_dict)
    
    dt =  pd.DataFrame.from_records([dict_all.values()], columns=dict_all.keys())
    dt['fid'] = fid
    dt['dim'] = dim
    return dt

In [ ]:
def get_all_info_bbob(fid, iid, dim):
    ela_dict = dict(dt_ela[dt_ela['fid'] == f"F{fid}_I{iid}_{dim}D"][relevant_columns].iloc[0])
    weight_dict = {f'W{idx}' : float(idx == fid-1) for idx in range(24)}
    iid_values = {f'I{idx}' : iid for idx, x in enumerate(iids.iloc[fid])}
    opt_loc = {f'x{idx}' : x for idx, x in enumerate(ioh.get_problem(fid, iid, dim).optimum.x)}
    dt_perf_sub = dt_perf[(dt_perf['kind'] == 'BBOB') & (dt_perf['fid'] == fid) & (dt_perf['dim'] == dim) & (dt_perf['iid'] == iid)]
    perf_dict = {x:y for x,y in zip(dt_perf_sub['ID'], dt_perf_sub['auc'])}

    dict_all = {}
    dict_all.update(ela_dict)
    dict_all.update(weight_dict)
    dict_all.update(iid_values)
    dict_all.update(opt_loc)
    dict_all.update(perf_dict)
    
    dt =  pd.DataFrame.from_records([dict_all.values()], columns=dict_all.keys())
    dt['fid'] = fid
    dt['dim'] = dim
    return dt

In [ ]:
dt_all_2d = pd.DataFrame()
for fid in range(1000):
    dt_all_2d = dt_all_2d.append(get_all_info_affine(fid, 2))
for fid in range(1,25):
    for iid in range(1,6):
        dt_all_2d = dt_all_2d.append(get_all_info_bbob(fid, iid, 2))


In [ ]:
dt_all_5d = pd.DataFrame()
for fid in range(1000):
    dt_all_5d = dt_all_5d.append(get_all_info_affine(fid, 5))
for fid in range(1,25):
    for iid in range(1,6):
        dt_all_5d = dt_all_5d.append(get_all_info_bbob(fid, iid, 5))


In [ ]:
dt_all_2d.to_csv("All_2d_info2.csv")
dt_all_5d.to_csv("All_5d_info2.csv")

### Load

In [ ]:
dt_all_2d = pd.read_csv("All_2d_info.csv", index_col=0)
dt_all_5d = pd.read_csv("All_5d_info.csv", index_col=0)

In [ ]:
algs = ['DiagonalCMA', 'DifferentialEvolution', 'modcma',
       'modde', 'RCobyla']

In [ ]:
best_algs = [algs[x] for x in np.array(dt_all_2d[algs]).argmax(axis=1)]
dt_all_2d['best'] = best_algs
best_algs = [algs[x] for x in np.array(dt_all_5d[algs]).argmax(axis=1)]
dt_all_5d['best'] = best_algs

### ELA plot distribution (Violins)

In [ ]:
ela_features = ['ela_meta.lin_simple.adj_r2', 'ela_meta.lin_simple.intercept',
       'ela_meta.lin_simple.coef.min', 'ela_meta.lin_simple.coef.max',
       'ela_meta.lin_simple.coef.max_by_min', 'ela_meta.lin_w_interact.adj_r2',
       'ela_meta.quad_simple.adj_r2', 'ela_meta.quad_simple.cond',
       'ela_meta.quad_w_interact.adj_r2', 'ela_distr.skewness',
       'ela_distr.kurtosis', 'ela_distr.number_of_peaks',
       'ela_level.mmce_lda_10', 'ela_level.mmce_qda_10',
       'ela_level.lda_qda_10', 'ela_level.mmce_lda_25',
       'ela_level.mmce_qda_25', 
       'ela_level.mmce_lda_50', 'ela_level.mmce_qda_50',
       'ela_level.lda_qda_50',  'nbc.nn_nb.sd_ratio',
       'nbc.nn_nb.mean_ratio', 'nbc.nn_nb.cor', 'nbc.dist_ratio.coeff_var',
       'nbc.nb_fitness.cor', 'disp.ratio_mean_02', 'disp.ratio_mean_05',
       'disp.ratio_mean_10', 'disp.ratio_mean_25', 'disp.ratio_median_02',
       'disp.ratio_median_05', 'disp.ratio_median_10', 'disp.ratio_median_25',
       'disp.diff_mean_02', 'disp.diff_mean_05', 'disp.diff_mean_10',
       'disp.diff_mean_25', 'disp.diff_median_02', 'disp.diff_median_05',
       'disp.diff_median_10', 'disp.diff_median_25', 'ic.h_max', 'ic.eps_s',
       'ic.m0']

In [ ]:
dt_temp = dt_all_5d[ela_features + ['kind', 'fid', 'dim']]

In [ ]:
dt_temp[ela_features] = (dt_temp[ela_features] - dt_temp[ela_features].min()) / ( dt_temp[ela_features].max() -  dt_temp[ela_features].min())

In [ ]:
dt_molten = dt_temp.melt(id_vars=['kind', 'fid', 'dim'])

In [ ]:
dt_molten

In [ ]:
plt.figure(figsize=(32,9))
sbs.violinplot(dt_molten, x='variable', y='value', hue='kind', split=True, cut=0, inner='quartile')
plt.ylim(0,1)
plt.xticks(rotation=30, fontsize=14)
plt.tight_layout()
plt.savefig("Figures/ELA_Feature_violins_5D_normalized.pdf")

### ELA plot (UMAP)

In [ ]:
def create_ELA_Map(dim, hue_var = 'kind', marker_var = 'kind', normalization=None):
    np.random.seed(42)
    if dim == 2:
        dt_dim = copy(dt_all_2d)
    else:
        dt_dim = copy(dt_all_5d)
    if normalization == 'normalize':
        dt_dim[ela_features] = (dt_dim[ela_features] - dt_dim[ela_features].min()) / ( dt_dim[ela_features].max() -  dt_dim[ela_features].min())
    if normalization == 'standardize':
        dt_dim[ela_features] = (dt_dim[ela_features] - dt_dim[ela_features].mean()) / dt_dim[ela_features].std()
    um = UMAP(random_state=42)
    um.fit(X=np.array(dt_dim[dt_dim['kind'] == 'BBOB'][ela_features]))
    
    transformed = um.transform(dt_dim[ela_features])
    
    dt_plot = pd.DataFrame(transformed, columns=['x0','x1'])
    dt_plot[hue_var] = np.array(dt_dim[hue_var])
    dt_plot[marker_var] = np.array(dt_dim[marker_var])

    plt.figure(figsize=(16,9))
    g = sbs.scatterplot(dt_plot, x='x0', y='x1', hue=hue_var, style=marker_var, s=450, linewidth=0.1, alpha=0.8)
    
    
    for lh in g.legend_.legendHandles: 
        lh.set_alpha(1)
        lh._sizes = [450] 
    
    
    plt.tight_layout()
    plt.savefig(f"Figures/ELA_UMAP_{dim}D_{normalization}_{hue_var}.pdf")

In [ ]:
for i in range(24):
    create_ELA_Map(5, normalization=None, hue_var=f'W{i}')

In [ ]:
for dim in [2,5]:
    for norm in [None, 'normalize', 'standardize']:
        for hue_var in ['best', 'kind']:
            create_ELA_Map(dim, normalization=norm, hue_var=hue_var)

### RF models

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
import sklearn

In [ ]:
def get_rf_score(dim, target, input_vars = 'ela', normalization = None):
    if dim == 2:
        dt_dim = copy(dt_all_2d)
    else:
        dt_dim = copy(dt_all_5d)
    if normalization == 'normalize':
        dt_dim[ela_features] = (dt_dim[ela_features] - dt_dim[ela_features].min()) / ( dt_dim[ela_features].max() -  dt_dim[ela_features].min())
    if normalization == 'standardize':
        dt_dim[ela_features] = (dt_dim[ela_features] - dt_dim[ela_features].mean()) / dt_dim[ela_features].std()
    
    if input_vars == 'ela':
        X = np.array(dt_dim[ela_features])
    else:
        X = np.array(dt_dim[[f'W{x}' for x in range(24)]])

    Y = np.array(dt_dim[target])
    if target == 'best':
        rf_model = RandomForestClassifier()
        scoring='f1_weighted'
    else:
        rf_model = RandomForestRegressor()
        scoring='neg_mean_absolute_error'
    
    
    res = cross_val_score(rf_model, X, Y, cv=10, scoring=scoring)
    return np.mean(res)

In [ ]:
items = []
for dim in [2,5]:
    for target in ['best'] + algs:
        for input_var in ['ela', 'weights']:
            res = get_rf_score(dim, target, input_var)
            items.append([res, dim, target, input_var])

In [ ]:
dt_rf_overall = pd.DataFrame.from_records(items, columns=['res', 'dim', 'target', 'input_var'])

In [ ]:
def get_rf_score_generalize(dim, target, input_vars = 'ela'):
    if dim == 2:
        dt_dim = copy(dt_all_2d)
    else:
        dt_dim = copy(dt_all_5d)
    
    if input_vars == 'ela':
        X = np.array(dt_dim[ela_features])
    else:
        X = np.array(dt_dim[[f'W{x}' for x in range(24)]])

    Y = np.array(dt_dim[target])
    if target == 'best':
        rf_model = RandomForestClassifier()
        scoring=partial(f1_score, average='weighted')
    else:
        rf_model = RandomForestRegressor()
        scoring=mean_absolute_error
    
    rf_model.fit(X[dt_dim['kind'] == 'BBOB'], Y[dt_dim['kind'] == 'BBOB'])
    preds = rf_model.predict(X[dt_dim['kind'] == 'Affine'])
    
    return scoring(preds, Y[dt_dim['kind'] == 'Affine'])
    

In [ ]:
from sklearn.metrics import mean_absolute_error, f1_score

In [ ]:
items = []
for dim in [2,5]:
    for target in ['best'] + algs:
        for input_var in ['ela', 'weights']:
            res = get_rf_score_generalize(dim, target, input_var)
            items.append([res, dim, target, input_var])

In [ ]:
dt_rf_generalize = pd.DataFrame.from_records(items, columns=['generalization', 'dim', 'target', 'input_var'])

In [ ]:
dt_combined_rf = dt_rf_generalize.merge(dt_rf_overall).reset_index()

In [ ]:
dt_combined_rf['res'] = np.abs(dt_combined_rf['res'])

In [ ]:
dt_combined_rf[dt_combined_rf['target'] == 'best']

In [ ]:
dt_combined_rf[dt_combined_rf['target'] == 'best']

In [ ]:
dt_table = dt_combined_rf[dt_combined_rf['target'] != 'best']

In [ ]:
res_table = np.zeros((4,10))

In [ ]:
for idx1, input_var in enumerate(['ela', 'weights']):
    for idx2, method in enumerate(['overall', 'generalize']):
        for idx3, alg in enumerate(algs):
            for idx4, dim in enumerate([2, 5]):
                val = dt_table[(dt_table['dim'] == dim) & (dt_table['target'] == alg) 
                               & (dt_table['input_var'] == input_var)]
                res_table[idx1+2*idx2, idx3+5*idx4] = val[['res', 'generalization'][idx2]]

In [ ]:
algs_short = ['dCMA', 'DE', 'modCMA', 'modDE', 'Cobyla']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
im = sbs.heatmap(res_table, ax=ax, vmin=0, vmax = 0.3, cbar_kws = dict(pad=0.01), cmap='viridis')
ax.set_yticks([x+0.5 for x in range(4)], ['ELA', 'Weights', 'ELA', 'Weights'], fontsize=20, rotation=0)
ax.set_xticks([0.5+x for x in range(10)], algs_short + algs_short, fontsize=23, rotation=-30)
plt.axhline(2, c='k', lw=4, ls='--')
plt.axvline(5, c='k', lw=4, ls='--')
plt.tight_layout()
plt.savefig("Figures/RF_MAE_algs_ELA_vs_Weights.pdf")

### Algorithm selector loss

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score

In [ ]:
def get_rf_losses(dim, target, version = 'generalize', input_vars = 'ela'):
    if dim == 2:
        dt_dim = copy(dt_all_2d)
    else:
        dt_dim = copy(dt_all_5d)
    
    if input_vars == 'ela':
        X = np.array(dt_dim[ela_features])
    else:
        X = np.array(dt_dim[[f'W{x}' for x in range(24)]])

    Y = np.array(dt_dim[target])
    if target == 'best':
        rf_model = RandomForestClassifier()
        scoring=partial(f1_score, average='weighted')
    else:
        rf_model = RandomForestRegressor()
        scoring=mean_absolute_error
    
    if version == 'generalize':
        rf_model.fit(X[dt_dim['kind'] == 'BBOB'], Y[dt_dim['kind'] == 'BBOB'])
        preds = rf_model.predict(X[dt_dim['kind'] == 'Affine'])
        dt_part = dt_dim[dt_dim['kind'] == 'Affine']
    else:
        preds = cross_val_predict(rf_model, X, Y, cv=10)
        dt_part = dt_dim
    # print(preds)
    pred_vals = np.array([dt_part.iloc[x][y] for x,y in enumerate(preds)])
    vbs_val = np.max(dt_part[algs], axis=1)
    
    return  vbs_val - pred_vals
    

In [ ]:
rf_model = RandomForestClassifier()

In [ ]:
preds = cross_val_predict(rf_model, X, Y, cv=10)

In [ ]:
pred_vals = [dt_all_2d.iloc[x][y] for x,y in enumerate(preds)]

In [ ]:
vbs_val = np.mean(np.max(dt_all_2d[algs], axis=1))

In [ ]:
np.mean(pred_vals) - vbs_val

In [ ]:
items = []
for dim in [2,5]:
    # for target in ['best'] + algs:
    for input_var in ['ela', 'weights']:
        for version in ['generalize', 'cv']:
            res = get_rf_losses(dim, 'best', version, input_var)
            for r in res:
                items.append([r, dim, 'best', input_var, version])

In [ ]:
dt_rf_loss = pd.DataFrame.from_records(items, columns=['loss', 'dim', 'target', 'input_var', 'version'])

In [ ]:
plt.figure(figsize=(16,9))
sbs.violinplot(dt_rf_loss, x='loss', y='version', hue='input_var', split=True, cut=0, inner='quartile')
plt.legend(title='Feature Used')
plt.tight_layout()
plt.savefig("Figures/AS_Model_losses.pdf")

In [ ]:
plt.figure(figsize=(16,9))
sbs.ecdfplot(dt_rf_loss, x='loss', hue=dt_rf_loss[['input_var', 'version']].apply(tuple, axis=1), lw=5, alpha=0.8)#, ls='dim')
# plt.legend(title='Feature Used')
plt.ylim(0.4,1)
plt.tight_layout()
plt.savefig("Figures/AS_Model_losses_cumulative2.pdf")